In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210313.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice]
0,09/03/2021 17:59:51,Samurai I: Musashi Miyamoto (1954),La Grande Vadrouille (1966),Dick (1999),NaN,MR BEAN-A-LIKES,ANOTHER DOCUMENTARY!,THE BLACK MARKET,[Default to Nicolas Cage]
1,09/03/2021 18:02:23,Dick (1999),What We Do in the Shadows (2014),Samurai I: Musashi Miyamoto (1954),La Grande Vadrouille (1966),ANOTHER DOCUMENTARY!,[Default to Nicolas Cage],THE BLACK MARKET,MR BEAN-A-LIKES
2,09/03/2021 18:12:54,La Grande Vadrouille (1966),What We Do in the Shadows (2014),Samurai I: Musashi Miyamoto (1954),Dick (1999),ANOTHER DOCUMENTARY!,MR BEAN-A-LIKES,THE BLACK MARKET,[Default to Nicolas Cage]
3,09/03/2021 18:16:57,La Grande Vadrouille (1966),What We Do in the Shadows (2014),Samurai I: Musashi Miyamoto (1954),Dick (1999),ANOTHER DOCUMENTARY!,MR BEAN-A-LIKES,[Default to Nicolas Cage],THE BLACK MARKET
4,09/03/2021 18:29:01,La Grande Vadrouille (1966),Samurai I: Musashi Miyamoto (1954),Dick (1999),What We Do in the Shadows (2014),THE BLACK MARKET,ANOTHER DOCUMENTARY!,[Default to Nicolas Cage],MR BEAN-A-LIKES
5,10/03/2021 09:34:09,Dick (1999),What We Do in the Shadows (2014),La Grande Vadrouille (1966),Samurai I: Musashi Miyamoto (1954),MR BEAN-A-LIKES,THE BLACK MARKET,ANOTHER DOCUMENTARY!,[Default to Nicolas Cage]


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Dick (1999)', 'What We Do in the Shadows (2014)',
       'Samurai I: Musashi Miyamoto (1954)',
       'La Grande Vadrouille (1966)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Dick (1999)': <Candidate('Dick (1999)')>,
 'What We Do in the Shadows (2014)': <Candidate('What We Do in the Shadows (2014)')>,
 'Samurai I: Musashi Miyamoto (1954)': <Candidate('Samurai I: Musashi Miyamoto (1954)')>,
 'La Grande Vadrouille (1966)': <Candidate('La Grande Vadrouille (1966)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Samurai I: Musashi Miyamoto (1954), La Grande Vadrouille (1966), Dick (1999))>,
 <Ballot(Dick (1999), What We Do in the Shadows (2014), Samurai I: Musashi Miyamoto (1954), La Grande Vadrouille (1966))>,
 <Ballot(La Grande Vadrouille (1966), What We Do in the Shadows (2014), Samurai I: Musashi Miyamoto (1954), Dick (1999))>,
 <Ballot(La Grande Vadrouille (1966), What We Do in the Shadows (2014), Samurai I: Musashi Miyamoto (1954), Dick (1999))>,
 <Ballot(La Grande Vadrouille (1966), Samurai I: Musashi Miyamoto (1954), Dick (1999), What We Do in the Shadows (2014))>,
 <Ballot(Dick (1999), What We Do in the Shadows (2014), La Grande Vadrouille (1966), Samurai I: Musashi Miyamoto (1954))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

FINAL RESULT
Candidate                             Votes  Status
----------------------------------  -------  --------
La Grande Vadrouille (1966)               3  Elected
Dick (1999)                               2  Rejected
Samurai I: Musashi Miyamoto (1954)        1  Rejected
What We Do in the Shadows (2014)          0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

FINAL RESULT
Candidate                    Votes  Status
-------------------------  -------  --------
ANOTHER DOCUMENTARY!             3  Elected
MR BEAN-A-LIKES                  2  Rejected
THE BLACK MARKET                 1  Rejected
[Default to Nicolas Cage]        0  Rejected

